In [12]:
import os, glob
import pandas as pd
from helper import *
import datetime
import numpy as np
pwd = os.path.dirname(os.path.realpath('.'))
data_folder = pwd + '/Data'
data_2018 = data_folder + '/2018'

In [3]:
list_of_speed_files_2018 = glob.glob(data_2018+'/speed_for_chunks/*')
list_of_csv_files_2018 = glob.glob(data_2018+'/2018_results/*')
annotations = pd.read_excel(data_2018+'/annotation.xlsx')
len(list_of_csv_files_2018)

4948

In [4]:
#  filter unvalid files
annotations = annotations[annotations.is_valid == 'Y']

In [5]:
name_of_csv_files_2018 = [i.split('/')[-1].split('.')[0] for i in list_of_csv_files_2018]
valid_files = [i for i in annotations.filename]
cnt = 0
for i in valid_files:
    if i in name_of_csv_files_2018:
        cnt += 1
print(cnt)
assert cnt == len(valid_files), "Not all the files found!"

462


In [91]:
cnt = 0

for j in annotations.filename:
    for i in name_of_csv_files_2018:
        if i == "unit01_2018-10-17_15_03_17_15_83064":
            file = [k for k in list_of_csv_files_2018 if k.split('/')[-1].split('.')[0] == j][0]
            df_detection = cluster(file)
            file = [k for k in list_of_speed_files_2018 if k.split('/')[-1].split('.')[0][:-6] == j][0]
            df_raw_speed = pd.read_csv(file)
            df_raw_speed.columns = ['ST_unit', 'Latitude', 'Longitude', 'Date', 'Time_UTC', 'newDateTime', 'Speed_knots', 'Speed_MPH', 'Speed_mps', 'valid']
            df_raw_speed['datetime'] = [datetime.datetime.strptime(str(i), '%H:%M:%S') for i in df_raw_speed.Time_UTC]
            df_raw_speed['datetime_time'] = df_raw_speed.datetime.dt.time
            df_detection['datetime_time'] = [i.round('1s').time() for i in pd.date_range(min(df_raw_speed.datetime), max(df_raw_speed.datetime), len(df_detection))]
            df_detection = pd.merge(df_detection,df_raw_speed[['datetime_time','Speed_MPH']],on='datetime_time', how='left')
            df_detection['filename'] = i
            df_detection.Speed_MPH.interpolate(inplace = True)
            df_detection = df_detection[df_detection.obj_ID == 1]
            df_detection = pd.merge(df_detection,annotations[[
                'filename','Type of vehicle','Type of road']], on='filename', how='left')
            df_detection = df_detection[['frame_num', 'dist', 'center', 'datetime_time',
                                        'Speed_MPH', 'filename', 'Type of vehicle', 'Type of road']]
            

            cnt += 1
    if cnt == 5:
        break